## Datenbereinigung

### Ziel der Datenbereinigung

Daten ***sauber***, ***konsistent*** und ***zuverlässig*** machen, ohne neue Informationen zu erzeugen.



In [1]:
# notwendige Bibliotheken importieren
import pandas as pd
import numpy as np

Link zu dem Datensatz [hier](https://zindi.africa/competitions/xente-fraud-detection-challenge) .

### EINLESEN

In [2]:
df = pd.read_csv("../data/raw/training.csv")
df.head(10)


FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/training.csv'

### Entfernen nicht-informativer Identifikationsspalten

Im Rahmen des Preprocessings wurden mehrere Identifikationsspalten aus dem Datensatz entfernt.  
Diese Spalten dienen ausschließlich der eindeutigen Zuordnung von Transaktionen bzw. Entitäten
und enthalten keine generalisierbare Information für das Lernproblem.

Folgende Spalten wurden aus dem Datensatz entfernt:

- `TransactionId`
- `BatchId`
- `AccountId`
- `SubscriptionId`
- `CustomerId`

Da diese Attribute eine sehr hohe Kardinalität aufweisen (nahezu eindeutige Werte pro Zeile),
würden sie beim Training eines Modells lediglich zu Overfitting führen, ohne einen inhaltlichen
Beitrag zur Betrugserkennung zu leisten.  
Die Entfernung erfolgt vor dem Train/Test-Split, da es sich hierbei um einen nicht-lernenden
Preprocessing-Schritt handelt.


In [4]:
DROP_COLS = ["TransactionId", "BatchId", "AccountId", "SubscriptionId", "CustomerId"]

DROP_COLS = [c for c in DROP_COLS if c in df.columns]

df = df.drop(columns=DROP_COLS)

df.head(10)


,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0
5,UGX,256,ProviderId_6,ProductId_3,airtime,ChannelId_3,2000.0,2000,2018-11-15T03:35:10Z,2,0
6,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,10000.0,10000,2018-11-15T03:44:31Z,4,0
7,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-500.0,500,2018-11-15T03:45:13Z,2,0
8,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,500.0,500,2018-11-15T04:14:59Z,2,0
9,UGX,256,ProviderId_1,ProductId_15,financial_services,ChannelId_3,600.0,600,2018-11-15T04:31:48Z,2,0


### Missing Values analysieren und Behandlungsstrategie festlegen

Nach dem Entfernen der nicht-informativen Identifikationsspalten wird als nächster Preprocessing-Schritt die
Vollständigkeit der Daten geprüft. Fehlende Werte können die Modellqualität sowie die Stabilität des Trainings
negativ beeinflussen und müssen daher **vor Encoding, Scaling und Modelltraining** systematisch behandelt werden.

Zunächst wird für jede Variable der **Anteil fehlender Werte** bestimmt. Auf Basis dieser Analyse wird eine
geeignete Imputationsstrategie festgelegt:

- **Numerische Merkmale** werden typischerweise mit dem **Median** imputiert, da dieser robuster gegenüber
  Ausreißern ist als der Mittelwert.

- **Kategoriale Merkmale** werden entweder mit dem **häufigsten Wert** imputiert oder erhalten eine eigene
  Kategorie (z. B. `"Missing"`), falls das Fehlen selbst eine informative Bedeutung haben kann.

- **Merkmale mit sehr hohem Missing-Anteil** werden kritisch geprüft und gegebenenfalls vollständig entfernt,
  sofern keine sinnvolle Imputation möglich ist.

Die eigentliche Imputation erfolgt später **innerhalb einer Pipeline**. Dabei werden die Imputer ausschließlich
auf dem Trainingsdatensatz gefittet, um **Data Leakage** zu vermeiden.


In [5]:

df = df.copy()

# 1) Missing-Rate pro Spalte berechnen
missing_ratio = df.isna().mean().sort_values(ascending=False)
missing_table = (missing_ratio * 100).to_frame("Missing_%")

display(missing_table)

# 2) Optional: nur Spalten anzeigen, die überhaupt Missing Values haben
missing_only = missing_table[missing_table["Missing_%"] > 0]
display(missing_only)




,Missing_%
CurrencyCode,0.0
CountryCode,0.0
ProviderId,0.0
ProductId,0.0
ProductCategory,0.0
ChannelId,0.0
Amount,0.0
Value,0.0
TransactionStartTime,0.0
PricingStrategy,0.0


,Missing_%


### Prüfung auf Duplikate

Im Rahmen der Datenbereinigung wird geprüft, ob der Datensatz doppelte Beobachtungen enthält.
Duplikate können durch Fehler bei der Datenerfassung oder beim Datenexport entstehen und führen
zu einer Verzerrung statistischer Kennzahlen sowie des Modelltrainings.

Daher wird zunächst die Anzahl vollständig identischer Zeilen bestimmt. Falls Duplikate
vorhanden sind, werden diese entfernt, sodass jede Transaktion im Datensatz nur einmal
repräsentiert ist.


In [6]:
# Anzahl komplett doppelter Zeilen
dup_rows = df.duplicated().sum()
print("Anzahl doppelter Zeilen:", dup_rows)

# Optional: Duplikate anzeigen (wenn vorhanden)
if dup_rows > 0:
    display(df[df.duplicated(keep=False)].head(20))
    df = df.drop_duplicates().reset_index(drop=True)
    print("Neues Shape nach Entfernen der Duplikate:", df.shape)


Anzahl doppelter Zeilen: 206


,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
94,UGX,256,ProviderId_6,ProductId_3,airtime,ChannelId_3,1000.0,1000,2018-11-15T07:03:26Z,2,0
95,UGX,256,ProviderId_6,ProductId_3,airtime,ChannelId_3,1000.0,1000,2018-11-15T07:03:26Z,2,0
96,UGX,256,ProviderId_6,ProductId_3,airtime,ChannelId_3,1000.0,1000,2018-11-15T07:03:26Z,2,0
97,UGX,256,ProviderId_6,ProductId_3,airtime,ChannelId_3,1000.0,1000,2018-11-15T07:03:26Z,2,0
98,UGX,256,ProviderId_6,ProductId_3,airtime,ChannelId_3,1000.0,1000,2018-11-15T07:03:26Z,2,0
99,UGX,256,ProviderId_6,ProductId_3,airtime,ChannelId_3,1000.0,1000,2018-11-15T07:03:26Z,2,0
100,UGX,256,ProviderId_6,ProductId_3,airtime,ChannelId_3,1000.0,1000,2018-11-15T07:03:26Z,2,0
101,UGX,256,ProviderId_6,ProductId_3,airtime,ChannelId_3,1000.0,1000,2018-11-15T07:03:26Z,2,0
102,UGX,256,ProviderId_6,ProductId_3,airtime,ChannelId_3,1000.0,1000,2018-11-15T07:03:26Z,2,0
103,UGX,256,ProviderId_6,ProductId_3,airtime,ChannelId_3,1000.0,1000,2018-11-15T07:03:27Z,2,0


Neues Shape nach Entfernen der Duplikate: (95456, 11)


### Plausibilitätsprüfung numerischer Merkmale

Nach dem Entfernen von Duplikaten werden numerische Merkmale auf Plausibilität und konsistente
Wertebereiche geprüft. Ziel ist es, potenzielle Erfassungs- oder Exportfehler (z. B. negative
Beträge, unrealistische Extremwerte oder inkonsistente Beziehungen zwischen Variablen) zu
identifizieren.

Im Fokus stehen insbesondere `Amount` und `Value`, da diese Merkmale direkt den Transaktionsbetrag
repräsentieren. Auffälligkeiten werden dokumentiert und anschließend wird entschieden, ob Werte
korrigiert, entfernt oder als fachlich zulässig beibehalten werden.


In [7]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

display(df[numeric_cols].describe(percentiles=[.01, .05, .5, .95, .99]).T)

# 2) Spezifische Checks für Amount & Value (falls vorhanden)
for col in ["Amount", "Value"]:
    if col in df.columns:
        print(f"\n--- Check für {col} ---")
        print("Min:", df[col].min())
        print("Max:", df[col].max())
        print("Anzahl = 0:", (df[col] == 0).sum())
        print("Anzahl < 0:", (df[col] < 0).sum())

# 3) Konsistenzcheck: Value sollte oft dem Betrag entsprechen (häufig |Amount|)
#    (je nach Definition kann Value = abs(Amount) sein)
if "Amount" in df.columns and "Value" in df.columns:
    diff = (df["Value"] - df["Amount"].abs())
    print("\n--- Konsistenzcheck Value vs. |Amount| ---")
    print("Anzahl Fälle, wo Value != |Amount|:", (diff != 0).sum())
    display(df.loc[diff != 0, ["Amount", "Value"]].head(20))


,count,mean,std,min,1%,5%,50%,95%,99%,max
CountryCode,95456.0,256.000000,0.000000,256.0,256.0,256.0,256.0,256.0,256.0,256.0
Amount,95456.0,6722.119086,123439.337807,-1000000.0,-30000.0,-5000.0,1000.0,14500.0,80000.0,9880000.0
Value,95456.0,9911.071436,123254.267808,2.0,10.0,25.0,1000.0,25000.0,90000.0,9880000.0
PricingStrategy,95456.0,2.256328,0.733138,0.0,1.0,2.0,2.0,4.0,4.0,4.0
FraudResult,95456.0,0.002022,0.044920,0.0,0.0,0.0,0.0,0.0,0.0,1.0



--- Check für Amount ---
Min: -1000000.0
Max: 9880000.0
Anzahl = 0: 0
Anzahl < 0: 38180

--- Check für Value ---
Min: 2
Max: 9880000
Anzahl = 0: 0
Anzahl < 0: 0

--- Konsistenzcheck Value vs. |Amount| ---
Anzahl Fälle, wo Value != |Amount|: 2560


,Amount,Value
3,20000.00,21800
34,-67.25,68
41,10000.00,11200
48,5000.00,5750
67,5000.00,5750
147,500.00,1115
158,5000.00,7000
222,17000.00,18710
226,-35.60,36
431,18500.00,20255


### Identifikation konstanter Merkmale

Nach der Plausibilitätsprüfung numerischer Merkmale wird untersucht, ob der Datensatz
konstante Merkmale enthält. Konstante Merkmale weisen über alle Beobachtungen hinweg
denselben Wert auf und besitzen somit keine Varianz.

Da solche Merkmale keine diskriminative Information enthalten, werden sie zunächst
identifiziert und dokumentiert. Erst anschließend wird entschieden, ob sie aus dem
Datensatz entfernt werden.


In [8]:
# Anzahl unterschiedlicher Werte pro Spalte berechnen
nunique = df.nunique(dropna=False)

# Konstante Merkmale identifizieren (genau ein eindeutiger Wert)
constant_cols = nunique[nunique == 1]

print("Konstante Merkmale (Spalten mit nur einem eindeutigen Wert):")
display(constant_cols.to_frame(name="Anzahl eindeutiger Werte"))

if not constant_cols.empty:
    df = df.drop(columns=constant_cols.index)
    print("Neues Shape nach Entfernen konstanter Merkmale:", df.shape)
    
df.head(10)


Konstante Merkmale (Spalten mit nur einem eindeutigen Wert):


,Anzahl eindeutiger Werte
CurrencyCode,1
CountryCode,1


Neues Shape nach Entfernen konstanter Merkmale: (95456, 9)


,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0
5,ProviderId_6,ProductId_3,airtime,ChannelId_3,2000.0,2000,2018-11-15T03:35:10Z,2,0
6,ProviderId_5,ProductId_3,airtime,ChannelId_3,10000.0,10000,2018-11-15T03:44:31Z,4,0
7,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-500.0,500,2018-11-15T03:45:13Z,2,0
8,ProviderId_6,ProductId_10,airtime,ChannelId_3,500.0,500,2018-11-15T04:14:59Z,2,0
9,ProviderId_1,ProductId_15,financial_services,ChannelId_3,600.0,600,2018-11-15T04:31:48Z,2,0


## Feature Engineering

### Ziel des Feature Engineerings

Variablen erstellen, umwandeln oder kodieren, um dem Modell zu helfen, Muster zu lernen.

### 1. Verarbeitung von Zeitstempel-Attributen

Zeitstempel-Attribute können von klassischen Machine-Learning-Modellen nicht direkt verarbeitet werden und werden daher im Rahmen des Feature Engineerings in aussagekräftige numerische Merkmale transformiert, um zeitliche Muster im Transaktionsverhalten abzubilden.

Aus dem Attribut `TransactionStartTime` werden zunächst klassische zeitbasierte Merkmale wie die Stunde des Tages, der Wochentag sowie Indikatoren für Wochenenden und Nachtzeiten extrahiert.

**Motivation für zyklische Kodierung**   
Zeitliche Merkmale wie **Stunden** oder **Wochentage** besitzen eine **zyklische Struktur**:
- Stunde 23 und Stunde 0 sind zeitlich benachbart,
- ebenso Sonntag und Montag. 

Eine lineare Kodierung würde diese zyklische Beziehung nicht korrekt abbilden. Um diesem Problem entgegenzuwirken, werden zyklische Merkmale mithilfe von **Sinus- und Kosinus-Transformationen** kodiert.
Diese Transformation projiziert die Zeitpunkte auf einen Kreis und erhält so ihre natürliche zyklische Struktur.

**Umsetzung**
- `ts_hour_sin` und `ts_hour_cos` kodieren die Tageszeit
- `ts_dow_sin` und `ts_dow_cos` kodieren den Wochentag 

In [9]:
import numpy as np
import pandas as pd

# Verarbeitung von Zeitstempel-Attributen

TIME_COL = "TransactionStartTime"

# Konvertierung in Datetime (UTC)
df[TIME_COL] = pd.to_datetime(df[TIME_COL], errors="coerce", utc=True)

# Klassische zeitbasierte Merkmale
df["ts_hour"] = df[TIME_COL].dt.hour
df["ts_dayofweek"] = df[TIME_COL].dt.dayofweek
df["ts_month"] = df[TIME_COL].dt.month

df["ts_is_weekend"] = df["ts_dayofweek"].isin([5, 6]).astype(int)
df["ts_is_night"] = df["ts_hour"].isin(range(0, 6)).astype(int)

# Zyklische Kodierung (SIN / COS)

# Stunde des Tages (0–23)
df["ts_hour_sin"] = np.sin(2 * np.pi * df["ts_hour"] / 24)
df["ts_hour_cos"] = np.cos(2 * np.pi * df["ts_hour"] / 24)

# Wochentag (0–6)
df["ts_dow_sin"] = np.sin(2 * np.pi * df["ts_dayofweek"] / 7)
df["ts_dow_cos"] = np.cos(2 * np.pi * df["ts_dayofweek"] / 7)

# -------------------------------
# Entfernen redundanter Spalten
# -------------------------------

df = df.drop(columns=[
    TIME_COL,
    "ts_hour",
    "ts_dayofweek"
])

df.head()


,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,PricingStrategy,FraudResult,ts_month,ts_is_weekend,ts_is_night,ts_hour_sin,ts_hour_cos,ts_dow_sin,ts_dow_cos
0,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2,0,11,0,1,0.500000,0.866025,0.433884,-0.900969
1,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2,0,11,0,1,0.500000,0.866025,0.433884,-0.900969
2,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2,0,11,0,1,0.500000,0.866025,0.433884,-0.900969
3,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2,0,11,0,1,0.707107,0.707107,0.433884,-0.900969
4,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2,0,11,0,1,0.707107,0.707107,0.433884,-0.900969


### 2. Identifikation numerischer und kategorialer Merkmale

Nach der Transformation zeitbasierter Merkmale werden die verbleibenden Features in
numerische und kategoriale Attribute unterteilt. Diese Trennung dient der Vorbereitung
einer strukturierten und reproduzierbaren Modell-Pipeline, in der unterschiedliche
Vorverarbeitungsschritte (z. B. Imputation, Encoding, Scaling) gezielt angewendet werden.

Die Zielvariable (`FraudResult`) wird hierbei explizit von den Eingangsmerkmalen getrennt,
um Data Leakage zu vermeiden.


In [10]:
# Definition der Feature-Gruppen für die Modell-Pipeline
TARGET = "FraudResult"

# PricingStrategy ist eine kategoriale Variable (laut Daten-Dictionary)
df["PricingStrategy"] = df["PricingStrategy"].astype(str)

categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
numeric_cols = [
    c for c in df.select_dtypes(include=["number"]).columns
    if c != TARGET
]

print(f"Numerische Features: {len(numeric_cols)}")
print("Numerische Spalten:")
print(numeric_cols)

print(f"Kategoriale Features: {len(categorical_cols)}")
print("Kategoriale Spalten:")
print(categorical_cols)


Numerische Features: 9
Numerische Spalten:
['Amount', 'Value', 'ts_month', 'ts_is_weekend', 'ts_is_night', 'ts_hour_sin', 'ts_hour_cos', 'ts_dow_sin', 'ts_dow_cos']
Kategoriale Features: 5
Kategoriale Spalten:
['ProviderId', 'ProductId', 'ProductCategory', 'ChannelId', 'PricingStrategy']


### 3. Betragsbezogenes Feature Engineering (Amount & Value)

Finanzielle **Transaktionsbeträge** weisen häufig stark schiefe **Verteilungen** und **Ausreißer** auf.
Um diese Effekte zu reduzieren und gleichzeitig inkonsistente Transaktionen identifizierbar
zu machen, werden zusätzliche betragsbezogene Merkmale erzeugt.

Hierzu werden:
- eine **Log-Transformation** der absoluten Transaktionswerte zur Stabilisierung der Skala,
- sowie **ein Verhältnismerkmal** zur Überprüfung der Konsistenz zwischen `Amount` und `Value`
erstellt.

Die ursprünglichen Merkmale bleiben erhalten, da sie weiterhin relevante Information für
die Modellierung enthalten.


In [11]:
import numpy as np

# Log-Transformation zur Reduktion von Schiefe und Ausreißereffekten
df["log_value"] = np.log1p(df["Value"])

# Verhältnismerkmal zur Identifikation inkonsistenter Transaktionen
df["amount_value_ratio"] = df["Amount"].abs() / (df["Value"] + 1)
df.head(10)

,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,PricingStrategy,FraudResult,ts_month,ts_is_weekend,ts_is_night,ts_hour_sin,ts_hour_cos,ts_dow_sin,ts_dow_cos,log_value,amount_value_ratio
0,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2,0,11,0,1,0.500000,0.866025,0.433884,-0.900969,6.908755,0.999001
1,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2,0,11,0,1,0.500000,0.866025,0.433884,-0.900969,3.044522,0.952381
2,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2,0,11,0,1,0.500000,0.866025,0.433884,-0.900969,6.216606,0.998004
3,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2,0,11,0,1,0.707107,0.707107,0.433884,-0.900969,9.989711,0.917389
4,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2,0,11,0,1,0.707107,0.707107,0.433884,-0.900969,6.469250,0.998450
5,ProviderId_6,ProductId_3,airtime,ChannelId_3,2000.0,2000,2,0,11,0,1,0.707107,0.707107,0.433884,-0.900969,7.601402,0.999500
6,ProviderId_5,ProductId_3,airtime,ChannelId_3,10000.0,10000,4,0,11,0,1,0.707107,0.707107,0.433884,-0.900969,9.210440,0.999900
7,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-500.0,500,2,0,11,0,1,0.707107,0.707107,0.433884,-0.900969,6.216606,0.998004
8,ProviderId_6,ProductId_10,airtime,ChannelId_3,500.0,500,2,0,11,0,1,0.866025,0.500000,0.433884,-0.900969,6.216606,0.998004
9,ProviderId_1,ProductId_15,financial_services,ChannelId_3,600.0,600,2,0,11,0,1,0.866025,0.500000,0.433884,-0.900969,6.398595,0.998336


### Speicherung des verarbeiteten Trainingsdatensatzes

Der bereinigte und feature-engineerte Trainingsdatensatz wird im Verzeichnis
`data/processed` gespeichert. Diese Version enthält ausschließlich deterministische
Transformationen und dient als stabile Grundlage für die nachfolgende Modellierung.


In [12]:
# Speicherung des vorverarbeiteten Datensatzes
df.to_csv("../data/processed/training_preprocessed.csv", index=False)